<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/%EC%97%B0%EC%8A%B5%EC%9A%A91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 159 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 7.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=1f0b30365d8d0b1dbbf458f17427ae0b551e26b4407a058793ee462b1330bc56
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 9.8 MB/s 
     |████████████████████████████████| 769 kB 8.7 MB/s 
     |████████████████████████████████| 895 kB 64.0 MB/s 
     |████████████████████████████████| 3.0 MB 50.7 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-rwyuqchl
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화  'Colab Notebooks'  'My Drive'


In [4]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'모델 개발용 자료'.zip -d /content/dataset
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/모델 개발용 자료.zip
2. 모델개발용자료.txt:  mismatching "local" filename (2. ыкиыН╕ъ░Ьы░ЬьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/2. 모델개발용자료.txt  
Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [5]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")
ts=pd.read_csv('/content/dataset/2. 모델개발용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [6]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
5,id_0000006,C,29,291,철,절삭.용접,카프라배관자재
6,id_0000007,I,56,561,음식점에서,접객시설을 갖추고,참치회(일본식)
7,id_0000008,C,10,107,쌀을 가지고,가공하여,떡제조
8,id_0000009,O,84,841,시청에서 재정과인력,일반공공행정,지방행정 집행
9,id_0000010,F,42,423,영업장에서,고객의뢰를 받아,내부전기공사


In [7]:
df['text']=df['text_obj']+" "+df['text_mthd']+" "+df['text_deal'] #text_obj+text_mthd+text_deal = text로 합치기
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000001,S,95,952,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,472,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,872,어린이집 보호자의 위탁을 받아 취학전아동보육
5,id_0000006,C,29,291,철 절삭.용접 카프라배관자재
6,id_0000007,I,56,561,음식점에서 접객시설을 갖추고 참치회(일본식)
7,id_0000008,C,10,107,쌀을 가지고 가공하여 떡제조
8,id_0000009,O,84,841,시청에서 재정과인력 일반공공행정 지방행정 집행
9,id_0000010,F,42,423,영업장에서 고객의뢰를 받아 내부전기공사


In [8]:
# 결측값 확인
df.isnull().sum()

AI_id          0
digit_1        0
digit_2        0
digit_3        0
text       74991
dtype: int64

In [9]:
df[(df['text'].isnull()== True)]

,AI_id,digit_1,digit_2,digit_3,text
10,id_0000011,C,10,102,NaN
21,id_0000022,R,91,911,NaN
29,id_0000030,C,29,292,NaN
40,id_0000041,I,55,551,NaN
46,id_0000047,I,55,551,NaN
...,...,...,...,...,...
999975,id_0999976,C,25,259,NaN
999976,id_0999977,G,46,467,NaN
999986,id_0999987,I,56,561,NaN
999987,id_0999988,I,55,551,NaN


In [10]:
df=df.dropna()
df.shape

(925009, 5)

In [11]:
df.isnull().sum() 

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

결측값이 있는 데이터가 제거되어 925009개 데이터만 남음

In [12]:
df['digit_1'].value_counts()

G    231309
I    167877
S     96995
H     95947
C     85000
P     45201
L     38371
Q     35128
F     32782
R     28169
M     25407
N     16335
K      9727
J      9539
O      2936
E      2201
A      1008
D       726
B       351
Name: digit_1, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [13]:
df.loc[(df['digit_1'] == "G"), 'digit_1'] = 0  
df.loc[(df['digit_1'] == "I"), 'digit_1'] = 1  
df.loc[(df['digit_1'] == "C"), 'digit_1'] = 2  
df.loc[(df['digit_1'] == "S"), 'digit_1'] = 3  
df.loc[(df['digit_1'] == "H"), 'digit_1'] = 4  
df.loc[(df['digit_1'] == "P"), 'digit_1'] = 5  
df.loc[(df['digit_1'] == "L"), 'digit_1'] = 6  
df.loc[(df['digit_1'] == "Q"), 'digit_1'] = 7  
df.loc[(df['digit_1'] == "F"), 'digit_1'] = 8  
df.loc[(df['digit_1'] == "R"), 'digit_1'] = 9  
df.loc[(df['digit_1'] == "M"), 'digit_1'] = 10  
df.loc[(df['digit_1'] == "N"), 'digit_1'] = 11 
df.loc[(df['digit_1'] == "J"), 'digit_1'] = 12  
df.loc[(df['digit_1'] == "K"), 'digit_1'] = 13  
df.loc[(df['digit_1'] == "O"), 'digit_1'] = 14  
df.loc[(df['digit_1'] == "E"), 'digit_1'] = 15  
df.loc[(df['digit_1'] == "A"), 'digit_1'] = 16 
df.loc[(df['digit_1'] == "D"), 'digit_1'] = 17  
df.loc[(df['digit_1'] == "B"), 'digit_1'] = 18  

In [14]:
data_list = []
for q, label in zip(df['text'], df['digit_1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['카센터에서 자동차부분정비 타이어오일교환', '3'], ['상점내에서 일반인을 대상으로 채소.과일판매', '0'], ['절단하여사업체에도매 공업용고무를가지고 합성고무도매', '0'], ['영업점에서 일반소비자에게 열쇠잠금장치', '0'], ['어린이집 보호자의 위탁을 받아 취학전아동보육', '7']]


## Train data & test data

In [16]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.9, random_state=0)

# KoBERT 입력 데이터로 만들기

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [18]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [19]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [21]:
data_train[0]

(array([   2, 4617, 5468, 6903, 3468, 6023, 7946, 7147, 7044, 7276, 6079,
        4617, 5468, 7344, 6093,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(16, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 7)

In [22]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [23]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=19,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [24]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [25]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1446 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.9999613761901855 train acc 0.0625
epoch 1 batch id 201 loss 1.1319326162338257 train acc 0.439443407960199
epoch 1 batch id 401 loss 0.4553035497665405 train acc 0.666341957605985
epoch 1 batch id 601 loss 0.18036486208438873 train acc 0.7553036605657238
epoch 1 batch id 801 loss 0.026761198416352272 train acc 0.8042681023720349
epoch 1 batch id 1001 loss 0.10790294408798218 train acc 0.8335882867132867
epoch 1 batch id 1201 loss 0.0669129490852356 train acc 0.8538847835137385
epoch 1 batch id 1401 loss 0.1965307891368866 train acc 0.86896636331192
epoch 1 train acc 0.8718879668049793


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 test acc 0.9723354748275966


  0%|          | 0/1446 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.20599210262298584 train acc 0.96875
epoch 3 batch id 201 loss 0.04501162841916084 train acc 0.9808768656716418
epoch 3 batch id 401 loss 0.068498894572258 train acc 0.9812967581047382
epoch 3 batch id 601 loss 0.010671206749975681 train acc 0.9825291181364393
epoch 3 batch id 801 loss 0.0770299956202507 train acc 0.9837507802746567
epoch 3 batch id 1001 loss 0.03624981269240379 train acc 0.9845623126873126
epoch 3 batch id 1201 loss 0.01502873282879591 train acc 0.9851035595337219
epoch 3 batch id 1401 loss 0.0061903889290988445 train acc 0.9857356352605282
epoch 3 train acc 0.9858121542185339


  0%|          | 0/13008 [00:00<?, ?it/s]

epoch 3 test acc 0.9748939965065634


# 문장 test 해보기

In [26]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append('G 도매 및 소매업(45~47)')
            elif np.argmax(logits) == 1:
                test_eval.append('I 숙박 및 음식점업(55~56)')
            elif np.argmax(logits) == 2:
                test_eval.append('C 제조업(10~34)')
            elif np.argmax(logits) == 3:
                test_eval.append('S 협회 및 단체, 수리 및 기타 개인 서비스업(94~96)')
            elif np.argmax(logits) == 4:
                test_eval.append('H 운수 및 창고업(49~52)')
            elif np.argmax(logits) == 5:
                test_eval.append('P 교육 서비스업(85)')
            elif np.argmax(logits) == 6:
                test_eval.append('L 부동산업(68)')
            elif np.argmax(logits) == 7:
                test_eval.append('Q 보건업 및 사회복지 서비스업(86~87)')
            elif np.argmax(logits) == 8:
                test_eval.append('F 건설업(41~42)')
            elif np.argmax(logits) == 9:
                test_eval.append('R 예술, 스포츠 및 여가관련 서비스업(90~91)')
            elif np.argmax(logits) == 10:
                test_eval.append('M 전문, 과학 및 기술 서비스업(70~73)')
            elif np.argmax(logits) == 11:
                test_eval.append('N 사업시설 관리, 사업 지원 및 임대 서비스업(74~76)')
            elif np.argmax(logits) == 12:
                test_eval.append('J 정보통신업(58~63)')
            elif np.argmax(logits) == 13:
                test_eval.append('K 금융 및 보험업(64~66)')
            elif np.argmax(logits) == 14:
                test_eval.append('O 공공 행정, 국방 및 사회보장 행정(84)')                               
            elif np.argmax(logits) == 15:
                test_eval.append('E 수도, 하수 및 폐기물 처리, 원료 재생업(36~39)')
            elif np.argmax(logits) == 16:
                test_eval.append('A 농업, 임업 및 어업(01~03)')
            elif np.argmax(logits) == 17:
                test_eval.append('D 전기, 가스, 증기 및 공기 조절 공급업(35)')
            elif np.argmax(logits) == 18:
                test_eval.append('B 광업(05~08)')
            
        return test_eval[0]

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

# 제출하기

In [27]:
ts['text']=ts['text_obj'].map(str)+" "+ts['text_mthd'].map(str)+" "+ts['text_deal'].map(str)

i=[]
for sentence in ts['text']:
    a=sentence.replace('nan', '')
    i.append(a)
ts['text']=pd.DataFrame(i)

ts=ts.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제

ts.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_000001,NaN,NaN,NaN,치킨전문점에서 고객의주문에의해 치킨판매
1,id_000002,NaN,NaN,NaN,산업공구 다른 소매업자에게 철물 수공구
2,id_000003,NaN,NaN,NaN,절에서 신도을 대상으로 불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서 고객요구로 자동차튜닝
4,id_000005,NaN,NaN,NaN,"실내포장마차에서 접객시설을 갖추고 소주,맥주제공"
5,id_000006,NaN,NaN,NaN,"철,아크릴,포맥스 스크린인쇄 명판"
6,id_000007,NaN,NaN,NaN,음식점 접객시설가지고 조개구이판매
7,id_000008,NaN,NaN,NaN,스테인레스를 프레스가공하여제조 주방용품
8,id_000009,NaN,NaN,NaN,수리 서비스센터에서 전문수리 수입차
9,id_000010,NaN,NaN,NaN,"약품(화공), 미싱 완성품입고, 수선"


In [ ]:
result=[]
for sentence in ts['text']:
  temp=predict(sentence)
  result.append(temp)

result

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
result